# Hikaru Nakamura Titled Tuesday Analysis

Titled Tuesday is Chess.com's weekly 11-round Swiss tournament for titled players, occurring twice each Tuesday since February 1, 2022. <br>
The analysis include data collected from chess.com API ([tutorial](https://www.chess.com/news/view/published-data-api#pubapi-endpoint-player-tournaments)) and span form 06/2015 to 09/2022

In [1]:
import pandas as pd
import requests
import datetime
import json

## Get all the games played

In [2]:
# the API divide the games in months
url = 'https://api.chess.com/pub/player/hikaru/games/archives'

# get all the months links
months = requests.get(url).content.decode()
months = json.loads(months)
months = months['archives']

In [3]:
months

['https://api.chess.com/pub/player/hikaru/games/2014/01',
 'https://api.chess.com/pub/player/hikaru/games/2014/02',
 'https://api.chess.com/pub/player/hikaru/games/2014/03',
 'https://api.chess.com/pub/player/hikaru/games/2014/04',
 'https://api.chess.com/pub/player/hikaru/games/2014/05',
 'https://api.chess.com/pub/player/hikaru/games/2014/06',
 'https://api.chess.com/pub/player/hikaru/games/2014/07',
 'https://api.chess.com/pub/player/hikaru/games/2014/08',
 'https://api.chess.com/pub/player/hikaru/games/2014/09',
 'https://api.chess.com/pub/player/hikaru/games/2014/10',
 'https://api.chess.com/pub/player/hikaru/games/2014/11',
 'https://api.chess.com/pub/player/hikaru/games/2014/12',
 'https://api.chess.com/pub/player/hikaru/games/2015/01',
 'https://api.chess.com/pub/player/hikaru/games/2015/02',
 'https://api.chess.com/pub/player/hikaru/games/2015/03',
 'https://api.chess.com/pub/player/hikaru/games/2015/04',
 'https://api.chess.com/pub/player/hikaru/games/2015/05',
 'https://api.

In [4]:
# let's see how are the game formatted
url = months[-1]

# get all the games
games = requests.get(url).content.decode()
games = json.loads(games)
games = games['games']

In [5]:
# the info we need are in the pgn 
games[0]['pgn'].split('\n')

['[Event "Live Chess"]',
 '[Site "Chess.com"]',
 '[Date "2022.09.01"]',
 '[Round "-"]',
 '[White "Hikaru"]',
 '[Black "TampaChess"]',
 '[Result "1-0"]',
 '[CurrentPosition "5knQ/2p2r1p/1p1p4/3P1qB1/4R2P/r4BP1/5P2/5RK1 b - -"]',
 '[Timezone "UTC"]',
 '[ECO "A00"]',
 '[ECOUrl "https://www.chess.com/openings/Kings-Fianchetto-Opening-1...e5-2.Bg2"]',
 '[UTCDate "2022.09.01"]',
 '[UTCTime "15:50:11"]',
 '[WhiteElo "3202"]',
 '[BlackElo "2821"]',
 '[TimeControl "180"]',
 '[Termination "Hikaru won by resignation"]',
 '[StartTime "15:50:11"]',
 '[EndDate "2022.09.01"]',
 '[EndTime "15:55:06"]',
 '[Link "https://www.chess.com/game/live/55786465275"]',
 '',
 '1. g3 {[%clk 0:03:00]} 1... e5 {[%clk 0:03:00]} 2. Bg2 {[%clk 0:02:59.5]} 2... Nc6 {[%clk 0:02:59.6]} 3. c4 {[%clk 0:02:49.7]} 3... g6 {[%clk 0:02:54.3]} 4. Nc3 {[%clk 0:02:49.6]} 4... Bg7 {[%clk 0:02:53.7]} 5. a3 {[%clk 0:02:48.9]} 5... d6 {[%clk 0:02:53.2]} 6. b4 {[%clk 0:02:48.2]} 6... a6 {[%clk 0:02:51.1]} 7. Rb1 {[%clk 0:02:47.3]} 7...

In [6]:
# we need a function for processing the data in a more digestible form

def better_form(game):
    # given the data as string it return them as a dictionarie
    dic = {}
    for info in game:
        key = info[1:info.index('"')-1] 
        value = info[info.index('"')+1:-2] # exploit that the 'content' is between "..."
        dic[key] = value
    return dic

In [7]:
all_games = []
for month in months:
    url = month
    games = requests.get(url).content.decode()
    games = json.loads(games)
    games = games['games']

    for game in games:
        try:
            # we will not need the moves for the analysis
            game = game['pgn'].split('\n')[:-3]
        except:
            # necessary since some games are different variation from the tradional chess games
            # and they do not have a pgn field
            continue
        all_games.append(better_form(game))

In [8]:
# convert the dictionaries in a dataframe
df = pd.DataFrame(all_games)
df

,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,ECO,...,TimeControl,Termination,StartTime,EndDate,EndTime,Link,SetUp,FEN,Variant,Tournament
0,Live Chess,Chess.com,2014.01.06,-,Hikaru,Godswill,1-0,6k1/1p2R3/p1p5/8/2P1B3/1P1P1p1P/P6K/8 b - -,UTC,C25,...,180,Hikaru won by resignation,23:50:17,2014.01.06,23:54:39,https://www.chess.com/game/live/692667823,NaN,NaN,NaN,NaN
1,Live Chess,Chess.com,2014.01.06,-,Godswill,Hikaru,0-1,8/8/7p/7P/1p6/pBkpp2b/8/4K3 w - -,UTC,A50,...,180,Hikaru won on time,23:54:43,2014.01.07,00:00:32,https://www.chess.com/game/live/692670646,NaN,NaN,NaN,NaN
2,Live Chess,Chess.com,2014.01.07,-,manueliux_06,Hikaru,0-1,rnb3k1/5ppp/2p5/p1b5/1p3B2/6P1/PPPNr1KP/R4R2 w...,UTC,C25,...,60,Hikaru won by resignation,00:34:20,2014.01.07,00:35:05,https://www.chess.com/game/live/692695595,NaN,NaN,NaN,NaN
3,Live Chess,Chess.com,2014.01.07,-,Hikaru,manueliux_06,1-0,8/p7/R6p/5pk1/8/6P1/P4P2/6K1 b - -,UTC,B10,...,60,Hikaru won by resignation,00:35:52,2014.01.07,00:37:45,https://www.chess.com/game/live/692696567,NaN,NaN,NaN,NaN
4,Live Chess,Chess.com,2014.01.07,-,Hikaru,Dmitriy_From_Russia,1-0,2R5/1p3rkp/1B1p1bp1/p2Pp3/P3B3/6PP/5P1K/8 b - -,UTC,B07,...,180,Hikaru won by resignation,00:43:10,2014.01.07,00:46:10,https://www.chess.com/game/live/692700868,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44866,Live Chess,Chess.com,2022.09.07,-,DesiAttack,Hikaru,0-1,4r1kr/p2n2pp/4Nn2/1P1b4/2BB4/2P5/P4P1P/R3K3 w Q -,UTC,A40,...,180,Hikaru won by resignation,17:53:12,2022.09.07,17:55:23,https://www.chess.com/game/live/56312115537,NaN,NaN,NaN,NaN
44867,Live Chess,Chess.com,2022.09.07,-,Hikaru,jcibarra,1-0,r1b2rk1/1p1nq1bp/p3N1p1/2ppP3/5PP1/1P2B1QP/P1P...,UTC,A00,...,180,Hikaru won by resignation,17:55:20,2022.09.07,17:56:27,https://www.chess.com/game/live/56312153875,NaN,NaN,NaN,NaN
44868,Live Chess,Chess.com,2022.09.07,-,Oleksandr_Bortnyk,Hikaru,0-1,2r1k2r/pb1qbp1p/1p1pp1p1/n7/P2PB3/B1P4Q/2PN1PP...,UTC,C00,...,180,Hikaru won by resignation,17:56:24,2022.09.07,17:58:16,https://www.chess.com/game/live/56312171467,NaN,NaN,NaN,NaN
44869,Live Chess,Chess.com,2022.09.10,-,NikoTheodorou,Hikaru,0-1,8/8/4pk2/1R4p1/5P2/8/1p4K1/1r6 w - -,UTC,A48,...,180,Hikaru won by resignation,00:54:15,2022.09.10,00:59:52,https://www.chess.com/game/live/56510364391,NaN,NaN,NaN,NaN


In [9]:
# we have in total almost 45000 games
# we dont need all the fields

df = df.drop(['CurrentPosition','ECOUrl', 'Round', 'Site', 'Event', 'Timezone', 'ECO', 'UTCDate', 'UTCTime', 'SetUp', 'FEN', 'Variant', 'Link'], axis = 1)
df

,Date,White,Black,Result,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Tournament
0,2014.01.06,Hikaru,Godswill,1-0,2354,2167,180,Hikaru won by resignation,23:50:17,2014.01.06,23:54:39,NaN
1,2014.01.06,Godswill,Hikaru,0-1,2163,2438,180,Hikaru won on time,23:54:43,2014.01.07,00:00:32,NaN
2,2014.01.07,manueliux_06,Hikaru,0-1,2104,2336,60,Hikaru won by resignation,00:34:20,2014.01.07,00:35:05,NaN
3,2014.01.07,Hikaru,manueliux_06,1-0,2411,2101,60,Hikaru won by resignation,00:35:52,2014.01.07,00:37:45,NaN
4,2014.01.07,Hikaru,Dmitriy_From_Russia,1-0,2509,2266,180,Hikaru won by resignation,00:43:10,2014.01.07,00:46:10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44866,2022.09.07,DesiAttack,Hikaru,0-1,2669,3209,180,Hikaru won by resignation,17:53:12,2022.09.07,17:55:23,NaN
44867,2022.09.07,Hikaru,jcibarra,1-0,3211,2822,180,Hikaru won by resignation,17:55:20,2022.09.07,17:56:27,NaN
44868,2022.09.07,Oleksandr_Bortnyk,Hikaru,0-1,3010,3215,180,Hikaru won by resignation,17:56:24,2022.09.07,17:58:16,NaN
44869,2022.09.10,NikoTheodorou,Hikaru,0-1,3007,3219,180,Hikaru won by resignation,00:54:15,2022.09.10,00:59:52,NaN


In [10]:
# we can save it
df.to_csv('all_games.csv', index = False)

## Focus on Titled Tuesday

In [11]:
# all the titled tuesday games will have an entry in the Tournement field
df = df.dropna()

In [12]:
# there are other tournament beside TT so we need to identify the one's we care
df['TT'] = df['Tournament'].apply(lambda x: 1 if x.find('titled-tuesday')!= -1 else 0)

C:\Users\matti\AppData\Local\Temp\ipykernel_10592\3842738628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TT'] = df['Tournament'].apply(lambda x: 1 if x.find('titled-tuesday')!= -1 else 0)


In [13]:
# we keep only the row that have 'titled-tuesday' substring in the Tournament field
df = df[df['TT'] == 1]

In [14]:
# we now have only 1645 games
df

,Date,White,Black,Result,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Tournament,TT
2140,2015.02.24,Hikaru,LADIESGET2KNOWME,1-0,2851,2302,180+2,Hikaru won on time,19:00:05,2015.02.24,19:07:11,https://www.chess.com/tournament/live/-titled-...,1
2141,2015.02.24,9ShMat2,Hikaru,0-1,2433,2852,180+2,Hikaru won by resignation,19:14:49,2015.02.24,19:24:51,https://www.chess.com/tournament/live/-titled-...,1
2142,2015.02.24,Hikaru,chito89,1/2-1/2,2847,2593,180+2,Game drawn by stalemate,19:26:27,2015.02.24,19:34:37,https://www.chess.com/tournament/live/-titled-...,1
2143,2015.02.24,megarompa,Hikaru,0-1,2641,2851,180+2,Hikaru won by resignation,19:38:48,2015.02.24,19:51:05,https://www.chess.com/tournament/live/-titled-...,1
2144,2015.02.24,Hikaru,GMNeiksans,1-0,2853,2522,180+2,Hikaru won on time,19:52:49,2015.02.24,20:03:34,https://www.chess.com/tournament/live/-titled-...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44836,2022.09.06,Orange_Ghost,Hikaru,0-1,2842,3212,180+1,Hikaru won by resignation,22:08:33,2022.09.06,22:13:12,https://www.chess.com/tournament/live/late-tit...,1
44837,2022.09.06,Dedic8,Hikaru,0-1,2863,3214,180+1,Hikaru won on time,22:18:34,2022.09.06,22:25:10,https://www.chess.com/tournament/live/late-tit...,1
44838,2022.09.06,Hikaru,penguingm1,1-0,3217,2924,180+1,Hikaru won on time,22:34:59,2022.09.06,22:42:10,https://www.chess.com/tournament/live/late-tit...,1
44839,2022.09.06,Sam_ChessMood,Hikaru,0-1,2888,3219,180+1,Hikaru won by resignation,22:44:40,2022.09.06,22:52:53,https://www.chess.com/tournament/live/late-tit...,1


In [15]:
# indentify all the TT
tournaments = df['Tournament']
tournaments = tournaments.drop_duplicates()
tournaments = tournaments.tolist()

In [16]:
# hikaru played in 170 TT
len(tournaments)

170

In [17]:
def get_results(row, i):
    # given a row of the df it compute if hikaru won, lost or drew and update the data structure
    if row['Result'] == '0-1':
        if row['White'] == 'Hikaru':
            tot_losses[i] += 1
        else:
            tot_wins[i] += 1
            
    elif row['Result'] == '1-0':
        if row['White'] == 'Hikaru':
            tot_wins[i] += 1
        else:
            tot_losses[i] += 1
            
    else:
        tot_draws[i] += 1

In [18]:
tot_wins = [0 for i in range(11)]
tot_losses = [0 for i in range(11)]
tot_draws = [0 for i in range(11)]

for tournament in tournaments:
    temp = df[df['Tournament'] == tournament]
    for i in range(len(temp)):
        get_results(temp.iloc[i], i)

In [19]:
# wins for every round
tot_wins

[159, 143, 136, 134, 115, 108, 104, 97, 93, 59, 55]

In [20]:
# losses for every round
tot_losses

[5, 14, 14, 12, 21, 16, 17, 22, 17, 24, 19]

In [21]:
# draws for every round
tot_draws

[6, 9, 16, 17, 23, 35, 35, 30, 33, 37, 20]

In [22]:
# we can check the number of games per round
for i in range(11):
    print(tot_wins[i]+tot_losses[i]+tot_draws[i])

# they decrease with every round (early quitting)
# note: round 10 and 11 were introduced later

170
166
166
163
159
159
156
149
143
120
94


In [23]:
# we now want to compute the opponent average rating for each round
def get_opponent_rating(row, i):
    # given a df row it return the opponent rating
    # it add it to the data struture we are using
    if row['White'] == 'Hikaru':
        rating[i] += int(row['BlackElo'])
    else:
        rating[i] += int(row['WhiteElo'])
    return rating

In [24]:
rating = [0 for i in range(11)]

for tournament in tournaments:
    # consider only tha games from the specific tournament
    temp = df[df['Tournament'] == tournament]
    
    for i in range(len(temp)):
        # games are altready ordered as the rounds
        rating = get_opponent_rating(temp.iloc[i], i)

for i in range(11):
    # to make the average we need to know the number of games played in the specific round
    tot = tot_wins[i] + tot_losses[i] + tot_draws[i]
    rating[i] = round(rating[i]/tot)

In [25]:
# average opponent rating for each round
rating

[2470, 2592, 2661, 2708, 2766, 2802, 2831, 2875, 2877, 2942, 2952]

## Other stats

### Record againts each opponent

In [26]:
white = df['White'].tolist()
black = df['Black'].tolist()
result = df['Result'].tolist()

In [27]:
# initialize all the opponent
# 'opponent' : [hikaru_won, hikaru_drew, hikaru_lost]
d = {}

for i in range(len(white)):
    if white[i] == 'Hikaru':
        d[black[i]] = [0,0,0]
    else:
        d[white[i]] = [0,0,0]

In [28]:
# in the 170 TT hikaru played against 752 different players
len(d)

752

In [29]:
for i in range(len(white)):
    if white[i] == 'Hikaru':
        if result[i] == '1-0':
            d[black[i]][0] += 1 # hikaru won
        elif result[i] == '0-1':
            d[black[i]][2] += 1 # hikaru lost
        else:
            d[black[i]][1] += 1 # hikaru drew
            
    else:
        if result[i] == '1-0':
            d[white[i]][2] += 1 # hikaru lost
        elif result[i] == '0-1': # hikaru won
            d[white[i]][0] += 1
        else:
            d[white[i]][1] += 1 # hikaru drew

In [30]:
d

{'LADIESGET2KNOWME': [1, 0, 0],
 '9ShMat2': [1, 1, 1],
 'chito89': [4, 1, 1],
 'megarompa': [1, 1, 0],
 'GMNeiksans': [2, 0, 0],
 'LyonBeast': [3, 3, 2],
 'tac49': [2, 0, 0],
 'GeorgMeier': [1, 2, 2],
 'amaidel': [1, 0, 0],
 'zmaj23': [1, 0, 1],
 'vovkcoach': [1, 0, 0],
 'Tilicheev_Viacheslav': [1, 1, 0],
 'Cryptochess': [4, 1, 0],
 'ChessTrener': [0, 1, 0],
 'Imre91': [2, 0, 0],
 'arnacman': [1, 0, 0],
 'Tal-Baronfew': [0, 1, 1],
 'kleinebeer98': [2, 0, 1],
 'DanielNaroditsky': [3, 5, 1],
 'Konavets': [2, 0, 0],
 'facu57': [2, 0, 0],
 'Dr-Bassem': [2, 0, 0],
 'Dlugy': [2, 1, 0],
 'lexiel': [1, 0, 0],
 'MinhGTrAn': [1, 0, 1],
 'nijat_a': [1, 1, 1],
 'VSERGUEI': [0, 1, 0],
 'KyronGriffith': [1, 0, 0],
 'GregShahade': [2, 0, 0],
 'Beca95': [4, 2, 1],
 'Mugzyyy': [1, 0, 0],
 'Tesla37': [1, 0, 0],
 'chessdude': [1, 0, 0],
 'Frolyanov_D': [1, 0, 0],
 'AndreyOstrovskiy': [1, 0, 0],
 'VMikhalevski': [2, 0, 0],
 'spectralsoul': [1, 0, 0],
 '1stSecond': [1, 0, 2],
 'jcibarra': [5, 2, 0],
 'GMha

### Opponent played the most

In [31]:
games_played = []
for key, value in d.items():
    games_played.append([sum(value), key])

sorted(games_played, reverse = True)
# hikaru played the most games against FairChess_on_YouTube, Oleksandr_Bortnyk and exoticprincess

[[30, 'FairChess_on_YouTube'],
 [28, 'Oleksandr_Bortnyk'],
 [24, 'exoticprincess'],
 [18, 'Jospem'],
 [17, 'wonderfultime'],
 [16, 'mishanick'],
 [15, 'jefferyx'],
 [15, 'Zhigalko_Sergei'],
 [14, 'TigrVShlyape'],
 [14, 'Bigfish1995'],
 [13, 'platy3'],
 [12, 'Msb2'],
 [11, 'rasmussvane'],
 [11, 'dropstoneDP'],
 [11, 'OparinGrigoriy'],
 [11, 'Duhless'],
 [10, 'erichansen'],
 [10, 'dretch'],
 [10, 'MITerryble'],
 [10, 'BogdanDeac'],
 [9, 'penguingm1'],
 [9, 'Zkid'],
 [9, 'Sanan_Sjugirov'],
 [9, 'Fandorine'],
 [9, 'DanielNaroditsky'],
 [8, 'joppie2'],
 [8, 'Parhamov'],
 [8, 'LyonBeast'],
 [8, 'GigaQuparadze'],
 [8, 'ChessWarrior7197'],
 [7, 'vladislavkovalev'],
 [7, 'jcibarra'],
 [7, 'champ2005'],
 [7, 'amintabatabaei'],
 [7, 'TimofeevAr'],
 [7, 'Shield12'],
 [7, 'Salem-AR'],
 [7, 'GMHess'],
 [7, 'Durarbayli'],
 [7, 'BilodeauA'],
 [7, 'BillieKimbah'],
 [7, 'Beca95'],
 [7, 'Anton_Demchenko'],
 [6, 'vugarrasulov'],
 [6, 'sergiochess83'],
 [6, 'kuli4ik'],
 [6, 'chito89'],
 [6, 'alexrustemov']

In [32]:
# record against FairChess_on_YouTube
# [won, drew, lost]
d[sorted(games_played, reverse = True)[0][1]]

[12, 12, 6]

In [33]:
# record against Oleksandr_Bortnyk
# [won, drew, lost]
d[sorted(games_played, reverse = True)[1][1]]

[17, 6, 5]

In [34]:
# record against exoticprincess
# [won, drew, lost]
d[sorted(games_played, reverse = True)[2][1]]

[11, 5, 8]

### Best and Worst record

In [35]:
games_played = []
for key, value in d.items():
    # define performace as wins - losses
    games_played.append([value[0]-value[2], key])

sorted(games_played, reverse = True)

[[13, 'platy3'],
 [12, 'Oleksandr_Bortnyk'],
 [10, 'OparinGrigoriy'],
 [10, 'Msb2'],
 [10, 'Jospem'],
 [9, 'TigrVShlyape'],
 [8, 'wonderfultime'],
 [8, 'Zhigalko_Sergei'],
 [7, 'mishanick'],
 [7, 'jefferyx'],
 [7, 'Zkid'],
 [7, 'Shield12'],
 [7, 'Fandorine'],
 [6, 'erichansen'],
 [6, 'alexrustemov'],
 [6, 'TimofeevAr'],
 [6, 'Reader777'],
 [6, 'GMHess'],
 [6, 'FairChess_on_YouTube'],
 [6, 'BilodeauA'],
 [6, 'BillieKimbah'],
 [6, 'Bigfish1995'],
 [6, 'Angry_Twin'],
 [5, 'penguingm1'],
 [5, 'jcibarra'],
 [5, 'amintabatabaei'],
 [5, 'Sanan_Sjugirov'],
 [5, 'Rodalquilar'],
 [5, 'GianmarcoVaR'],
 [5, 'GMBenjaminBok'],
 [5, 'Duhless'],
 [4, 'vladislavkovalev'],
 [4, 'shimastream'],
 [4, 'sergiochess83'],
 [4, 'seorobert1971'],
 [4, 'rasmussvane'],
 [4, 'oskariot'],
 [4, 'maciek_92'],
 [4, 'kuli4ik'],
 [4, 'dropstoneDP'],
 [4, 'bipe137'],
 [4, 'attack2mateU'],
 [4, 'WPWPWP-2015'],
 [4, 'TimurGareyev'],
 [4, 'Rud_Makarian'],
 [4, 'MITerryble'],
 [4, 'GM_Crest'],
 [4, 'DmitryMaximov'],
 [4, 'Cr

In [36]:
# best performace
d['platy3']

[13, 0, 0]

In [37]:
# worst performace
d['GigaQuparadze']

[2, 2, 4]

In [38]:
# let's dive dipper in this bad record
df[((df['White']=='Hikaru') & (df['Black']=='GigaQuparadze'))]

,Date,White,Black,Result,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Tournament,TT


In [39]:
# so hikaru never played as white against GigaQuparadze
df[((df['White']=='GigaQuparadze') & (df['Black']=='Hikaru'))]

,Date,White,Black,Result,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Tournament,TT
36329,2020.12.01,GigaQuparadze,Hikaru,0-1,2925,3191,180+1,Hikaru won by resignation,19:58:25,2020.12.01,20:04:15,https://www.chess.com/tournament/live/-titled-...,1
37235,2021.01.26,GigaQuparadze,Hikaru,1-0,2995,3240,180+1,GigaQuparadze won by resignation,19:50:44,2021.01.26,19:57:31,https://www.chess.com/tournament/live/-titled-...,1
37869,2021.03.09,GigaQuparadze,Hikaru,0-1,2964,3174,180+1,Hikaru won by resignation,19:14:25,2021.03.09,19:21:55,https://www.chess.com/tournament/live/-titled-...,1
38725,2021.05.04,GigaQuparadze,Hikaru,1-0,2953,3217,180+1,GigaQuparadze won by resignation,18:11:38,2021.05.04,18:18:45,https://www.chess.com/tournament/live/titled-t...,1
39627,2021.06.29,GigaQuparadze,Hikaru,1-0,2970,3153,180+1,GigaQuparadze won on time,17:45:06,2021.06.29,17:52:31,https://www.chess.com/tournament/live/titled-t...,1
39800,2021.07.13,GigaQuparadze,Hikaru,1/2-1/2,2966,3223,180+1,Game drawn by agreement,18:18:46,2021.07.13,18:24:36,https://www.chess.com/tournament/live/titled-t...,1
39857,2021.07.20,GigaQuparadze,Hikaru,1-0,2995,3257,180+1,GigaQuparadze won by resignation,18:45:53,2021.07.20,18:52:59,https://www.chess.com/tournament/live/titled-t...,1
40004,2021.07.27,GigaQuparadze,Hikaru,1/2-1/2,3015,3224,180+1,Game drawn by repetition,18:45:04,2021.07.27,18:52:34,https://www.chess.com/tournament/live/titled-t...,1


### Number of Stalemate

In [40]:
df['Stalemate'] = df['Termination'].apply(lambda x: 1 if x[-9:] == 'stalemate' else 0)

In [41]:
df[df['Stalemate'] == 1]

,Date,White,Black,Result,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Tournament,TT,Stalemate
2142,2015.02.24,Hikaru,chito89,1/2-1/2,2847,2593,180+2,Game drawn by stalemate,19:26:27,2015.02.24,19:34:37,https://www.chess.com/tournament/live/-titled-...,1,1
2923,2015.08.04,Beca95,Hikaru,1/2-1/2,2575,2850,180+2,Game drawn by stalemate,19:39:55,2015.08.04,19:52:43,https://www.chess.com/tournament/live/-titled-...,1,1
6156,2016.02.03,dretch,Hikaru,1/2-1/2,2464,2831,180+2,Game drawn by stalemate,01:12:47,2016.02.03,01:22:53,https://www.chess.com/tournament/live/-titled-...,1,1
13358,2017.06.06,Hikaru,maciek_92,1/2-1/2,2907,2534,180+2,Game drawn by stalemate,21:18:57,2017.06.06,21:27:34,https://www.chess.com/tournament/live/-titled-...,1,1
26886,2019.10.01,Hikaru,alishersuleymenov,1/2-1/2,3152,2712,180+2,Game drawn by stalemate,21:03:36,2019.10.01,21:12:26,https://www.chess.com/tournament/live/-titled-...,1,1
34688,2020.09.29,Indianlad,Hikaru,1/2-1/2,2948,3231,180+1,Game drawn by stalemate,18:48:24,2020.09.29,18:57:01,https://www.chess.com/tournament/live/-scc-gra...,1,1
37424,2021.02.16,Hikaru,IMRosen,1/2-1/2,3201,2610,180+1,Game drawn by stalemate,18:11:28,2021.02.16,18:20:59,https://www.chess.com/tournament/live/-titled-...,1,1
37754,2021.03.02,Hikaru,fearless_guy,1/2-1/2,3222,2856,180+1,Game drawn by stalemate,20:02:09,2021.03.02,20:11:36,https://www.chess.com/tournament/live/-titled-...,1,1
38189,2021.03.30,DanielNaroditsky,Hikaru,1/2-1/2,3051,3211,180+1,Game drawn by stalemate,18:10:34,2021.03.30,18:19:21,https://www.chess.com/tournament/live/-titled-...,1,1
38989,2021.05.18,Hikaru,ChessWarrior7197,1/2-1/2,3202,2930,180+1,Game drawn by stalemate,18:00:34,2021.05.18,18:09:03,https://www.chess.com/tournament/live/titled-t...,1,1


In [42]:
# hikaru had 14 stalemates in TT
len(df[df['Stalemate'] == 1])

14

### How many TT hikaru won + average number of players for each year

In [43]:
# we can get this kind of data this way

url_patter = 'https://api.chess.com/pub/tournament/' + tournaments[0].split('/')[-1]
data = requests.get(url_patter).content.decode()
data = json.loads(data)
data

{'name': '*** Titled Tuesday 3|2 Blitz',
 'url': 'https://www.chess.com/tournament/live/-titled-tuesday-32-blitz-492386',
 'creator': 'CHESScom',
 'status': 'finished',
 'start_time': 1424804400,
 'finish_time': 1424811809,
 'settings': {'type': 'swiss',
  'rules': 'chess',
  'is_rated': True,
  'is_official': False,
  'is_invite_only': False,
  'user_advance_count': 1,
  'winner_places': 3,
  'registered_user_count': 106,
  'total_rounds': 9,
  'time_class': 'blitz',
  'time_control': '180+2'},
 'players': [{'username': 'eltajsafarli', 'status': 'winner'},
  {'username': 'jcibarra', 'status': 'registered'},
  {'username': 'exoticprincess', 'status': 'registered'},
  {'username': 'lyonbeast', 'status': 'registered'},
  {'username': 'ar4uha', 'status': 'registered'},
  {'username': 'georgmeier', 'status': 'registered'},
  {'username': 'pedromartinez91', 'status': 'registered'},
  {'username': 'nouki', 'status': 'registered'},
  {'username': '9shmat2', 'status': 'registered'},
  {'userna

In [44]:
data_each_TT = []
for tournament in tournaments:
    tournament = tournament.split('/')
    url_patter = 'https://api.chess.com/pub/tournament/' + tournament[-1]
    
    data = requests.get(url_patter).content.decode()
    data = json.loads(data)
    
    won = 0
    for player_results in data['players']:
        if player_results['username'] == 'hikaru':
            if player_results['status'] == 'winner':
                won = 1

    data_each_TT.append([datetime.date.fromtimestamp(data['start_time']).year, data['settings']['registered_user_count'], won])


In [45]:
data_each_TT = pd.DataFrame(data_each_TT)
data_each_TT.columns = ['Year', 'Players', 'Won']

In [46]:
data_each_TT

,Year,Players,Won
0,2015,106,0
1,2015,95,1
2,2015,108,0
3,2015,83,1
4,2015,128,0
...,...,...,...
165,2022,295,1
166,2022,369,0
167,2022,315,0
168,2022,385,1


In [47]:
# average number of player each year
data_each_TT[['Year', 'Players']].groupby(by=['Year']).mean().astype('int')

,Players
Year,
2015,105
2016,108
2017,183
2018,309
2019,387
2020,783
2021,601
2022,363


In [48]:
# TT won by Hikaru per year
data_each_TT[['Year', 'Won']].groupby(by=['Year']).sum()

,Won
Year,
2015,3
2016,4
2017,2
2018,0
2019,1
2020,7
2021,12
2022,19


In [49]:
# in total Hikaru won 48 TT
data_each_TT[['Year', 'Won']].groupby(by=['Year']).sum().sum()

Won    48
dtype: int64